# Solution for Day 8: Treetop Tree House

*Advent of Code 2022*

- [Day 8 Challenge](https://adventofcode.com/2022/day/8)

- [Instructions](instructions.md)

- [Input Data](input.txt)

---

In [1]:
# imports
import numpy as np

In [2]:
INPUT_FILE = 'input.txt'

In [3]:
with open(INPUT_FILE, 'r') as f:
    lines = [line.strip('\n') for line in f.readlines()]
    
# lines

---

## Part I

### Create Rows and Columns from Input Data

In [19]:
# Change lines to a list of lists where each list contains numbers

def create_array(lines):
    """Create an array from a list of numeric strings."""
    rows = [[int(num) for num in list(line)] for line in lines]
    rows = np.array(rows)
    return rows

In [26]:
# Create rows of numbers from the input data
rows = create_array(lines)

# Check that the number of rows and columns in the columns array are equal
# rows.shape

In [27]:
# Now we can transpose the rows to "look down" columns as well
columns = np.transpose(rows)

# Check that the number of rows and columns in the columns array are equal
# columns.shape

### Count Trees on Outer Edges

In [14]:
def count_visible_edge_trees(rows, cols):
    """Counts the number of trees on the edges of the array."""
    
    # Ensure that the number of items in rows is the same as in columns.
    if len(rows[0]) == len(cols[0]):
        # The number of trees on the edge 
        outer_visible = (len(rows[0]) - 1) * 4
        
        return outer_visible
    
    # If rows and columns are not a true array, return an error message
    else:
        return f'Error: Length of rows ({len(rows)}) does not match length of columns ({len(columns)})'


In [15]:
count_visible_edge_trees(rows, columns)

392

### Create Array from Interior Trees

### Testcases

In [8]:
example_input = """30373
25512
65332
33549
35390
"""

example_lines = [line.strip('\n') for line in example_input.split()]

example_rows = create_array(example_lines)
example_cols = np.transpose(example_rows)

# outer = (len(example_rows[0]) - 1) * (len(example_cols[0] - 1))
# outer

def find_number_edge_trees(rows, cols):
    outer_visible = (len(rows[0]) - 1) * (len(cols[0]) - 1)
    return outer_visible

print(find_number_edge_trees(example_rows, example_cols))

16


### Answer

---

## Part II

### Answer